<a href="https://colab.research.google.com/github/BiancaStadl/ProjektarbeitML/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Quelle für Twitter-Daten: https://github.com/uds-lsv/GermEval-2018-Data/

Michael Wiegand, Melanie Siegel, and Josef Ruppenhofer: "Overview of the GermEval 2018 Shared Task on the Identification of Offensive Language", in Proceedings of the GermEval, 2018, Vienna, Austria.

In [31]:
#import matplotlib.pyplot as plt -> für evtl Visualisierungen
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np
from keras import losses
from tensorflow import keras 
from keras import optimizers 
from keras import metrics 

#!pip install Tokenizer
from keras.preprocessing.text import Tokenizer
#!pip install pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
#from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


Einige Parameter für das Netz setzen.. https://becominghuman.ai/creating-your-own-neural-network-using-tensorflow-fa8ca7cc4d0e 

In [32]:
training_epochs = 10
batch_size = 32
max_length = 60

In [33]:
url = "https://github.com/uds-lsv/GermEval-2018-Data/archive/master.zip"

dataset = tf.keras.utils.get_file("GermEval-2018-Data-master.zip", url, 
                                   extract=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'GermEval-2018-Data-master')


In [34]:
os.listdir(dataset_dir)

['submissions.tgz',
 'germeval2018.test.txt',
 'results.pdf',
 'germeval2018.training.txt',
 'guidelines-iggsa-shared.pdf',
 'evaluationScriptGermeval2018.pl',
 'LICENSE.md',
 'survey.pdf',
 'README.md']

In [35]:
training_file = os.path.join(dataset_dir, 'germeval2018.training.txt')
#with open(training_file) as f:
 # print(f.read())

#print()

testing_file = os.path.join(dataset_dir, 'germeval2018.test.txt')


In [36]:
def remove_clutter(string):
   string = re.sub("@[^\s]+"," ",string)
   string = re.sub("#[^\s]+"," ", string)
   string = re.sub("\u00a9"," ", string)
   string = re.sub("\u00ae"," ", string)
   string = re.sub("[\u2000-\u3300]"," ", string)
   string = re.sub("\ud83c[\ud000-\udfff]"," ", string)
   string = re.sub("\ud83d[\ud000-\udfff]"," ", string)
   string = re.sub("\ud83e[\ud000-\udfff]"," ", string)
   string = re.sub("😜", " ",string)
   string = re.sub("🍫", " ",string)
   string = re.sub("😁", " ",string)
   string = re.sub("🐖", " ",string)
   string = re.sub("😡", " ",string)
   string = re.sub("😇", " ",string)
   string = re.sub("😬", " ",string)
   string = re.sub("😃", " ",string)
   string = re.sub("😂", " ",string)
   string = re.sub("💙", " ",string)  
   string = re.sub("😛", " ",string)
   string = re.sub("🙏", " ",string)
   string = re.sub("👍", " ",string)
   string = re.sub("🖕", " ",string)
   string = re.sub("😉", " ",string)
   string = re.sub("💩", " ",string)
   string = re.sub("🤢", " ",string)
   string = re.sub("👏", " ",string)
   string = re.sub("😨", " ",string)
   string = re.sub("🤣", " ",string)
   string = re.sub("🤡", " ",string)
   string = re.sub("😈", " ",string)
   string = re.sub("💃🏽", " ",string)
   string = re.sub("👹", " ",string)
   string = re.sub("🤘", " ",string)
   string = re.sub("😱", " ",string)
   string = re.sub("🤔", " ",string) 
   string = re.sub("🌈", " ",string) 
   string = re.sub("💕", " ",string) 
   string = re.sub("👩‍❤️‍👩", " ",string) 
   string = re.sub("😍", " ",string) 
   string = re.sub("👆", " ",string) 
   string = re.sub("😖", " ",string) 
   string = re.sub("👇", " ",string) 
   string = re.sub("🔥", " ",string) 
   string = re.sub("😘", " ",string) 
   string = re.sub("🎉", " ",string) 
   string = re.sub("🤬", " ",string) 
   string = re.sub("👊", " ",string)
   string = re.sub("🇩🇪", " ",string)  
   
   string = re.sub("OTHER|OFFENSE|ABUSE|INSULT"," ",string)
   return string

In [37]:
statementsForTraining = []
sentimentsForTraining = []

fileToRead = open(training_file, 'r')

while True:
  #next line in file
  line = fileToRead.readline()

  if line == "":
   break

  findSentiment = re.search("OTHER|OFFENSE",line)

  line = remove_clutter(line)
      

  statementsForTraining.append(line)

   #sentimentsForTraining.append(findSentiment.group(0))

  if findSentiment.group(0) == "OTHER":  
    sentimentsForTraining.append(0)
  else:
    sentimentsForTraining.append(1)

  if not line:
    break

 #print("{}: {}".format(count,line.strip()))
  
 # print(sentiment.group(0))
 
fileToRead.close()

training_sentences = statementsForTraining
training_labels = sentimentsForTraining

#print(training_sentences[0:100])
#print(training_labels[9])  

In [38]:

statementsForTesting = []
sentimentsForTesting = []

fileToRead = open(testing_file, 'r')

while True:
 
  line = fileToRead.readline()

  if line == "":
   break

  sent = re.search("OTHER|OFFENSE",line)

  line = remove_clutter(line)


  statementsForTesting.append(line)
  #print(len(line))
  #sentimentsForTesting.append(sent.group(0))

  if sent.group(0) == "OTHER": 
    sentimentsForTesting.append(0)
  else:
    sentimentsForTesting.append(1)

  if not line:
    break


fileToRead.close()


testing_sentences = statementsForTesting
testing_labels = sentimentsForTesting
#print(len(testing_sentences))
#print(testing_sentences)   
#print(statementsForTesting)

In [39]:
tokenizer = Tokenizer(oov_token="OOV")
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index
validation_size = 500

training_sequences = tokenizer.texts_to_sequences(training_sentences)
padded_training = pad_sequences(training_sequences, maxlen=max_length, padding='post')

#print(len(padded_training))

validation_sequences = padded_training[0:validation_size]
validation_labels = training_labels[0:validation_size]

padded_training = padded_training[validation_size:]
training_labels = training_labels[validation_size:]

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
padded_testing = pad_sequences(testing_sequences, maxlen=max_length, padding='post')

#print(validation_sequences[499])
#print(padded_training[0])
#print(len(validation_labels))
#print(len(training_labels))


nppadded_training = np.array(padded_training)
nptraining_labels = np.array(training_labels)

nppadded_validation = np.array(validation_sequences)
npvalidation_labels = np.array(validation_labels)

nppadded_testing = np.array(padded_testing)
nptesting_labels = np.array(testing_labels)



In [40]:

#print(statementsForTraining[2])

print(validation_sequences[4])
print(statementsForTraining[4])
#print(nppadded_training.shape)

#print(nptraining_labels[4])
#print(nppadded_testing.shape)

print(word_index) 

[  93 5803 5804 5805   66   14 5806  851   60 5807    8    2 3633    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]
  kein verläßlicher Verhandlungspartner. Nachkarteln nach den Sondierzngsgesprächen - schickt diese Stümper   in die Versenkung.	 	 

{'OOV': 1, 'die': 2, 'der': 3, 'und': 4, 'das': 5, 'ist': 6, 'lbr': 7, 'in': 8, 'nicht': 9, 'ich': 10, 'es': 11, 'sie': 12, 'zu': 13, 'den': 14, 'für': 15, 'ein': 16, 'mit': 17, 'wir': 18, 'von': 19, 'auch': 20, 'sich': 21, 'sind': 22, 'auf': 23, 'hat': 24, 'nur': 25, 'eine': 26, 'was': 27, 'wie': 28, 'so': 29, 'im': 30, 'noch': 31, 'man': 32, 'aber': 33, 'haben': 34, 'als': 35, 'wird': 36, 'wenn': 37, 'werden': 38, 'bei': 39, 'an': 40, 'aus': 41, 'dem': 42, 'schon': 43, 'merkel': 44, 'ihr': 45, 'oder': 46, 'da': 47, 'dass': 48, 'keine': 49, 'uns': 50, '

In [41]:
from google.colab import drive
#drive.mount("/content/drive")
os.listdir("/content/drive/MyDrive/Colab Notebooks")

['germeval_training.txt',
 'glove.twitter.27B.50d.txt',
 'glove.twitter.27B.200d.txt',
 'glove.840B.300d.txt',
 'tensorboard.gdoc',
 'keras.gdoc',
 'LSTM_limited_vocab.ipynb',
 'Vectorization CNN embedded_limited_vocab.ipynb',
 'CNN embedded_limited_vocab.ipynb',
 'CNN embedded.ipynb',
 'LSTM.ipynb']

In [42]:
#Größe Vokabel -> wordindex + 2 (weil padding + OOV) 
hits = 0
misses = 0

vocabulary_size = len(word_index)+2

# dann erstell ich ein Wörterbuch mit Namen "embedding_vector", dort sind dann
#die keys drinnen, die in glove-Datei drinnen sind mit dem entsprechenden Key

embedding_index_glove = {}
f = open('/content/drive/MyDrive/Colab Notebooks/glove.twitter.27B.200d.txt')
for line in f:
  value = line.split()
  word = value[0]
  coef = np.asarray(value[1:],dtype='float32')
  embedding_index_glove[word] = coef

print("%d gefunden: "% len(embedding_index_glove))

#Dann noch eine Embedding-Matrix erstellen
#zweiter Wert = Embedding-Dimension der Datei, in dem Fall 200

glove_matrix = np.zeros((vocabulary_size,200))
for word, index in tokenizer.word_index.items():
    embedding_value = embedding_index_glove.get(word)
    if embedding_value is not None:
      glove_matrix[index] = embedding_value
      hits+=1
    else:
      misses+=1

print("hits %d and %d misses"%(hits,misses))

1193514 gefunden: 
hits 6747 and 8383 misses


In [43]:
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding, LSTM

output-dimension -> Werte variieren.. Zuerst 16, da war die Accuracy nur etwas über 60, bei LSTM 10.
Dann Output-dim auf 120 und LSTM auf 10 -> Accuracy fast bei 90%

dann LSTM auf 265 und dim auf 120 -> knapp über 60

In [44]:
import keras.backend as K

def metrics_recall(data_true, data_pred):
    data_true = K.ones_like(data_true)
    true_positives = K.sum(K.round(K.clip(data_true*data_pred,0,1)))
    possible_positives = K.sum(K.round(K.clip(data_true,0,1)))

    recall = true_positives / (possible_positives+K.epsilon())
    return recall


def metrics_precision(data_true, data_pred):
    data_true = K.ones_like(data_true)
    true_positives = K.sum(K.round(K.clip(data_true*data_pred,0,1)))

    positives_predicted = K.sum(K.round(K.clip(data_pred,0,1)))
    precision = true_positives / (positives_predicted+K.epsilon())
    return precision


def metrics_f1(data_true, data_pred):
    precision_data = metrics_precision(data_true, data_pred)
    recall_data = metrics_recall(data_true, data_pred)
    return 2*(precision_data*recall_data)/(precision_data+recall_data+K.epsilon())

In [45]:
from tensorflow.keras.layers import Embedding
print(len(word_index))
LSTM_model = tf.keras.Sequential()
#LSTM_model.add(tf.keras.layers.Embedding(input_dim=15132, output_dim=100, input_length=max_length))
LSTM_model.add(tf.keras.layers.Embedding(vocabulary_size, output_dim=200, input_length=60, embeddings_initializer = keras.initializers.Constant(glove_matrix), trainable= False))
LSTM_model.add(tf.keras.layers.Conv1D(filters=10, kernel_size=3,activation='relu'))
LSTM_model.add(tf.keras.layers.MaxPooling1D())
#LSTM_model.add(tf.keras.layers.LSTM(265, activation="tanh",recurrent_activation="sigmoid",recurrent_dropout=0,unroll="false",use_bias="True"))
LSTM_model.add(tf.keras.layers.LSTM(30, activation="tanh", recurrent_activation="sigmoid", use_bias=True, return_sequences=True))
#LSTM_model.add(tf.keras.layers.GlobalMaxPooling1D())
LSTM_model.add(tf.keras.layers.LSTM(30))
LSTM_model.add(tf.keras.layers.Dropout(0.5))
LSTM_model.add(tf.keras.layers.Dense(65,activation='relu'))
LSTM_model.add(tf.keras.layers.Dense(1))



15130


In [51]:
LSTM_model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy',metrics_recall,metrics_precision,metrics_f1])
print(LSTM_model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 60, 200)           3026400   
_________________________________________________________________
conv1d (Conv1D)              (None, 58, 10)            6010      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 29, 10)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 29, 30)            4920      
_________________________________________________________________
lstm_3 (LSTM)                (None, 30)                7320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 65)               

In [ ]:
LSTM_model.layers[1].get_weights()[0].shape

In [ ]:
%load_ext tensorboard

In [48]:
logs_base_dir = "./logs"
callbackForTB = tf.keras.callbacks.TensorBoard(logs_base_dir)

In [52]:
LSTM_model.fit(nppadded_training, nptraining_labels, batch_size=32, epochs=11, validation_data=(nppadded_testing, nptesting_labels), callbacks=[callbackForTB])

Epoch 1/11
141/141 [==============================] - 11s 52ms/step - loss: 0.8245 - accuracy: 0.6309 - metrics_recall: 0.0635 - metrics_precision: 0.7001 - metrics_f1: 0.1124 - val_loss: 0.6535 - val_accuracy: 0.6597 - val_metrics_recall: 0.0000e+00 - val_metrics_precision: 0.0000e+00 - val_metrics_f1: 0.0000e+00
Epoch 2/11
141/141 [==============================] - 6s 42ms/step - loss: 0.6386 - accuracy: 0.6545 - metrics_recall: 0.0212 - metrics_precision: 0.3341 - metrics_f1: 0.0390 - val_loss: 0.6278 - val_accuracy: 0.6594 - val_metrics_recall: 2.8153e-04 - val_metrics_precision: 0.0090 - val_metrics_f1: 5.4600e-04
Epoch 3/11
141/141 [==============================] - 6s 42ms/step - loss: 0.6215 - accuracy: 0.6713 - metrics_recall: 0.0673 - metrics_precision: 0.6187 - metrics_f1: 0.1135 - val_loss: 0.6267 - val_accuracy: 0.6602 - val_metrics_recall: 0.0158 - val_metrics_precision: 0.3784 - val_metrics_f1: 0.0300
Epoch 4/11
141/141 [==============================] - 6s 41ms/step - l

In [ ]:
%tensorboard --logdir {logs_base_dir}

In [ ]:
from tensorboard import notebook
notebook.list()

In [ ]:
notebook.display(port=6006, height=1000) 
#error..

In [ ]:
#!pip install tensorboardcolab
#from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback

#tbc=TensorBoardColab()

In [58]:
results = LSTM_model.evaluate(nppadded_testing, nptesting_labels, batch_size=batch_size)
#print("test loss, test acc:",results)

111/111 [==============================] - 1s 10ms/step - loss: 0.5934 - accuracy: 0.6911 - metrics_recall: 0.2466 - metrics_precision: 1.0000 - metrics_f1: 0.3897


In [53]:
LSTM_predict = LSTM_model.predict(nppadded_testing[200:210])

In [54]:
for p in LSTM_predict:
  print(p)

[0.27835533]
[0.461136]
[0.4413174]
[0.3646042]
[0.44543055]
[0.36873826]
[1.0043379]
[0.17626029]
[0.1889894]
[0.46039298]


In [55]:
prediction_rounded = np.argmax(LSTM_predict, axis=1)

for p in prediction_rounded:
  print(p)


print(nptesting_labels[200:210])

0
0
0
0
0
0
0
0
0
0
[0 1 0 1 0 0 1 0 0 0]


In [ ]:
#print(testing_labels[10:16])
#print(statementsForTesting[4:7])

In [56]:
tf.math.confusion_matrix(nptesting_labels[200:210], prediction_rounded)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[7, 0],
       [3, 0]], dtype=int32)>